# Monthly Reports

## Head FLEX Days Used

In [94]:
# start with imports
import pandas as pd
import db_functions as dbfnc
import config as cfg
from datetime import date

In [95]:
# grab data from the db and input it to a df
# the query is HeadworkedTable joined with ShowTable joined with HeadNamesTable filtered on ShowName 'FLEX DAY'
sql_file=open(f'{cfg.sql_dir}\head_toil_days.sql')
query = sql_file.read()

with dbfnc.connection(cfg.my_driver, cfg.my_server, cfg.my_db) as conn:
    df = pd.read_sql(query, conn)

In [96]:
# filter for last 24 months
value_to_check = pd.Timestamp(date.today().year-2, date.today().month, date.today().day)
filter_mask = df['ShiftDate'] > value_to_check
df = df[filter_mask]

In [97]:
# turn the datetime64 object into a string object for manipultaion
df['Date_String'] = df['ShiftDate'].dt.strftime('%Y-%m-%d')

In [98]:
# create the fiscal info from the new string object
df['Fiscal Year'] = df.Date_String.apply(dbfnc.fiscal_yr)
df['Fiscal Quarter'] = df.Date_String.apply(dbfnc.fiscal_q)

In [99]:
# sort for presentation
df.sort_values(by='FirstName', inplace=True)
sort_list = ['Fiscal Year', 'Fiscal Quarter']
df.sort_values(by=sort_list, ascending=False, inplace=True)

In [100]:
#create the new df removing uneeded column data
df = df[['Fiscal Year', 'Fiscal Quarter', 'FirstName', 'ShiftDate']].copy()

In [101]:
#.. and style for preentation
df.style.set_properties(**{'text-align':'center'}).hide_index()

Fiscal Year,Fiscal Quarter,FirstName,ShiftDate
2020-2021,2,Arran,2021-02-05 00:00:00
2020-2021,2,David,2021-01-29 00:00:00
2020-2021,2,Jack,2021-01-14 00:00:00
2020-2021,2,Jaclyn,2020-12-11 00:00:00
2020-2021,2,Kris,2021-02-19 00:00:00
2020-2021,2,Matt,2021-02-26 00:00:00
2020-2021,2,Rikki,2021-02-25 00:00:00
2020-2021,2,Terry,2021-02-17 00:00:00
2020-2021,1,Arran,2020-10-16 00:00:00
2020-2021,1,David,2020-11-20 00:00:00
